In [65]:
#!pip install rtmlib==0.0.13
#!pip install opencv-python==4.11.0.86
#!pip install librosa==0.11.0
#!pip install Cython==3.0.12
# Microsoft Visual C++ 14.0 or greater is required.
# Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
# The pip version of madmon is outdated and the github repo has many changes recently
#!pip uninstall -y madmom
#!conda install git+https://github.com/CPJKU/madmom.git
#!cd madmom && python setup.py develop --user
#!pip install pyaudio==0.2.14
#!pip install beatnet==1.1.1
#!pip install soundfile==0.13.1
# Install FFmpeg in Windows
# Source: https://phoenixnap.com/kb/ffmpeg-windows
#!pip cache purge
#!pip install moviepy==2.1.1
#!pip install onnxruntime==1.21.1

In [63]:
import os

move_filename = '88ae640'
moves_filepath = os.path.join('outputs', 'moves.json')

In [65]:
!ffmpeg -i inputs/videos/88ae640.mp4 -ab 160k -ac 2 -ar 44100 -vn processed/videos/88ae640.wav

ffmpeg version 2025-04-23-git-25b0a8e295-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (Rev3, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx

In [67]:
from BeatNet.BeatNet import BeatNet
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
audio_filepath = os.path.join('processed', 'videos', f'{move_filename}.wav')
estimator = BeatNet(1, mode='offline', inference_model='DBN', plot=[], thread=False)
beats = estimator.process(audio_filepath)
beats

array([[0.12, 4.  ],
       [0.56, 1.  ],
       [1.  , 2.  ],
       [1.44, 3.  ],
       [1.9 , 4.  ],
       [2.32, 1.  ],
       [2.78, 2.  ],
       [3.2 , 3.  ],
       [3.64, 4.  ]])

In [17]:
# 9da1eb8 = 4
# 5db13cd = 1
# 5d80c9a = 1
# 022dccf = - 1

In [69]:
offset = 0
# Fix the beat numbers
for index in range(len(beats)):
    beats[index][1] = int((index + offset) % 8)
#beats = beats[:-1]
beats

array([[0.12, 0.  ],
       [0.56, 1.  ],
       [1.  , 2.  ],
       [1.44, 3.  ],
       [1.9 , 4.  ],
       [2.32, 5.  ],
       [2.78, 6.  ],
       [3.2 , 7.  ],
       [3.64, 0.  ]])

In [73]:

offset = -1
beat_start = 0.0
beat_end = 3.64
num_beats = 9
beat_delta = (beat_end - beat_start) / (num_beats - 1)
beats = [[round(beat_start + x * beat_delta, 2), (x + offset) % 8] for x in range(num_beats)]
beats


[[0.0, 7],
 [0.46, 0],
 [0.91, 1],
 [1.36, 2],
 [1.82, 3],
 [2.27, 4],
 [2.73, 5],
 [3.19, 6],
 [3.64, 7]]

In [11]:
'''
# Fix the beat numbers
for index in range(12, 16):
    beats[index][1] = beats[index][1] - 4
beats
'''

'\n# Fix the beat numbers\nfor index in range(12, 16):\n    beats[index][1] = beats[index][1] - 4\nbeats\n'

In [75]:
from moviepy import VideoFileClip, TextClip, CompositeVideoClip

video_file = f"inputs/videos/{move_filename}.mp4"
output_file = f"processed/videos/{move_filename}_counts.mp4"


# Load the video
video_clip = VideoFileClip(video_file)

start_counting = True
text_clips = [video_clip]

for index, x in enumerate(beats):
    start_time = x[0]
    end_time = beats[index + 1][0] if index + 1 < len(beats) else x[0]

    start_counting = start_counting or x[1] == 1

    if start_counting:
        # Create the text clip
        text_clip = TextClip(font='arial.ttf', text=f'{int(x[1] + 1)}', font_size=70, color='white', duration=end_time - start_time)
        
        # Set the duration and position of the text clip
        text_clip = text_clip.with_position(('center', 'bottom'))
        
        # Set the start time of the text clip
        text_clip = text_clip.with_start(start_time)

        text_clips.append(text_clip)
        
# Composite the text clip on the video
video_clip = CompositeVideoClip(text_clips)

# Write the output video file
video_clip.write_videofile(output_file, fps=30, codec='libx264')

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf62.3.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 127, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'video', 'language': None, 'default': True, 'size': [720, 1280], 'bitrate': 8821, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc62.11.100 libx264'}}], 'input_number': 0}], 'duration': 3.67, 'bitrate': 8961, 'start': 0.0, 'default_audio_input_number': 0, 'default_audio_stream_number': 0, 'audio_fps': 44100, 'audio_bitrate': 127, 'default_video_input_number': 0, 'default_video_stream_number': 1, 'video_codec_na

MoviePy - Done.
MoviePy - Writing video processed/videos/88ae640_counts.mp4



MoviePy - Done !
MoviePy - video ready processed/videos/88ae640_counts.mp4


In [77]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

# After verifying that the counts are Okay, I can save the counts
data[move_filename] = []
section = 1
for index, x in enumerate(beats):
    count = int(x[1] + 1)
    print(count)
    data[move_filename].append({
        'section': section, 
        'count': count,
        'time': x[0],
        'leader_keypoints': [],
        'follower_keypoints': [],
        'offset': offset
    })
    if count == 8:
        section += 1

with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

8
1
2
3
4
5
6
7
8


In [79]:
import cv2

start_counting = False
section = 1
for index, x in enumerate(beats):
    count = int(x[1] + 1)
    print(count)
    vidcap = cv2.VideoCapture(video_file)
    vidcap.set(cv2.CAP_PROP_POS_MSEC, int(x[0] * 1000))      # just cue to position
    success, image = vidcap.read()
    if success:
        cv2.imwrite(os.path.join('processed', 'images', f"{move_filename}_section_{section}_count_{count}.jpg"), image)     # save frame as JPEG file
    else:
        print(f'Error in index: {index}')
        # data[move_filename].pop(index)
    # Release resources
    vidcap.release()
    cv2.destroyAllWindows()

    if count == 8 or (count == 4 and index < len(beats) - 2 and beats[index + 1][1] == 0):
        section += 1

8
1
2
3
4
5
6
7
8


In [83]:
import numpy as np
import cv2
import os
from rtmlib import Body, draw_skeleton
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

device = 'cpu'
backend = 'onnxruntime'  # opencv, onnxruntime, openvino

cap = cv2.VideoCapture(0)

openpose_skeleton = False  # True for openpose-style, False for mmpose-style

body = Body(
    pose='rtmo',
    to_openpose=openpose_skeleton,
    mode='performance',  # balanced, performance, lightweight
    backend=backend,
    device=device
)
# TODO: Nowadays, I do this by hand. We need to automate it.
leader_detected_first = [
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False
]
leader_detected_first = [False for index in range(len(beats))]
img_index = 0

for filename in os.listdir(os.path.join('processed', 'images')):
    # check if the image ends with png
    if (filename.startswith(move_filename)):
        frame = cv2.imread(os.path.join('processed', 'images', filename), cv2.IMREAD_COLOR)
        keypoints, scores = body(frame)

        if leader_detected_first[img_index]:
            data[move_filename][img_index]["leader_keypoints"] = keypoints[0].tolist()
            if keypoints.shape[0] > 1:
                data[move_filename][img_index]["follower_keypoints"] = keypoints[1].tolist()
        elif np.sum(keypoints) > 0:
            data[move_filename][img_index]["follower_keypoints"] = keypoints[0].tolist()
            if keypoints.shape[0] > 1:
                data[move_filename][img_index]["leader_keypoints"] = keypoints[1].tolist()
        print(filename)
        print(keypoints.shape)
        print(np.sum(keypoints))
        print(keypoints[0])

        img_show = frame.copy()
        img_show = draw_skeleton(
            img_show,
            keypoints,
            #np.array([keypoints[0]]),
            scores,
            openpose_skeleton=openpose_skeleton,
            kpt_thr=0.3,
            line_width=2
        )

        cv2.imwrite(os.path.join('processed', 'keypoints', filename), img_show)
        img_index += 1

# Override the data with the keypoints
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

load C:\Users\user\.cache\rtmlib\hub\checkpoints\rtmo-l_16xb16-600e_body7-640x640-b37118ce_20231211.onnx with onnxruntime backend
88ae640_section_1_count_8.jpg
(1, 17, 2)
16003.041
[[344.2301  461.69037]
 [343.01834 451.30646]
 [346.72464 452.5603 ]
 [325.65826 457.53275]
 [370.5587  459.70752]
 [304.32904 506.82074]
 [402.8038  520.02026]
 [246.95012 440.4442 ]
 [413.67117 575.4611 ]
 [232.0098  385.38333]
 [376.89645 562.81714]
 [315.05838 681.63885]
 [375.67654 682.10034]
 [333.2307  815.1222 ]
 [391.13672 811.9596 ]
 [346.33737 935.5321 ]
 [405.89703 928.75586]]
88ae640_section_2_count_1.jpg
(1, 17, 2)
15521.295
[[ 308.38187   408.12738 ]
 [ 299.00073   392.99243 ]
 [ 305.37323   399.88464 ]
 [ 254.49026   402.97775 ]
 [ 321.0229    416.37933 ]
 [ 234.34653   489.53284 ]
 [ 355.42627   507.45862 ]
 [ 151.19017   551.7422  ]
 [ 382.88657   611.1733  ]
 [  85.545074  613.7765  ]
 [ 313.0881    580.53986 ]
 [ 197.11461   727.18866 ]
 [ 288.66217   738.7301  ]
 [ 158.02426   931.82306 

In [85]:
import numpy as np

def normalize_keypoints_by_bbox_height(keypoints):
    """
    Normalize keypoints so that:
    - The top-left of the bounding box is (0, 0)
    - The bounding box height is 1
    """
    kp = np.array(keypoints)
    
    min_x = np.min(kp[:, 0])
    min_y = np.min(kp[:, 1])
    max_y = np.max(kp[:, 1])
    
    bbox_height = max_y - min_y
    if bbox_height < 1e-5:
        raise ValueError("Bounding box height too small for normalization.")
    
    # Translate to origin and scale
    kp_normalized = (kp - [min_x, min_y]) / bbox_height
    return kp_normalized, bbox_height


def compute_normalized_embeddings(leader_kp, follower_kp):
    leader_kp = np.array(leader_kp)
    follower_kp = np.array(follower_kp)

    # Normalize each person's keypoints by their own bbox height
    norm_leader = np.array([])
    norm_follower = np.array([])
    if np.sum(leader_kp) > 0:
        norm_leader, leader_height = normalize_keypoints_by_bbox_height(leader_kp)
    if np.sum(follower_kp) > 0:
        norm_follower, _ = normalize_keypoints_by_bbox_height(follower_kp)

    rel_vector = np.array([])
    if np.sum(leader_kp) > 0 and np.sum(follower_kp) > 0:
        # Get original centers (before normalization)
        center_leader = np.mean(leader_kp, axis=0)
        center_follower = np.mean(follower_kp, axis=0)
    
        # Compute relative position vector in leader's height units
        rel_vector = (center_follower - center_leader) / leader_height

    return norm_leader.flatten(), norm_follower.flatten(), rel_vector

In [87]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

for index in range(len(data[move_filename])):
    kp_leader = data[move_filename][index]['leader_keypoints']
    kp_follower = data[move_filename][index]['follower_keypoints']
    leader_embedding, follower_embedding, distance_embedding = compute_normalized_embeddings(kp_leader, kp_follower)
    data[move_filename][index]['leader_embedding'] = leader_embedding.tolist()
    data[move_filename][index]['follower_embedding'] = follower_embedding.tolist()
    data[move_filename][index]['distance_embedding'] = distance_embedding.tolist()

# Override the data with the new embeddings
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

In [89]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

# Add musicality (TODO: This is manual)
musicality = [
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '-'
]
for index in range(len(data[move_filename])):
    data[move_filename][index]['musicality'] = musicality[index]

# Override the data with musicality
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

In [101]:
import random
import numpy as np
import json
import os

moves_filepath = os.path.join('outputs', 'moves.json')
# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    moves_data = json.load(file)

# -------------------------
# 1. Build embeddings
# -------------------------
def build_embedding(move_item):
    """Return a 34+34+2 = 70 sized embedding"""

    l = move_item.get("leader_embedding", [])
    f = move_item.get("follower_embedding", [])
    d = move_item.get("distance_embedding", [])

    # pad or slice to fixed sizes
    l = (l + [0]*34)[:34]
    f = (f + [0]*34)[:34]
    d = (d + [0]*2)[:2]

    return np.array(l + f + d, dtype=float)

# -------------------------
# 2. Cosine Similarity
# -------------------------
def cosine_sim(a, b):
    if np.all(a == 0) or np.all(b == 0):
        return 0
    return np.dot(a, b) / (np.linalg.norm(a)*np.linalg.norm(b))

embedding_map = {x + 1: [] for x in range(8)}
for move_id in moves_data:
    current_move = moves_data[move_id]
    for index, move_item in enumerate(current_move):
        current_count = move_item['count']
        embedding_map[move_item['count']].append({
            'move_id': move_id,
            'index': index,
            'embedding': build_embedding(move_item)
        })

for count, items in embedding_map.items():
    for index, item in enumerate(items):
        current_embedding = item['embedding']
        embedding_candidates = [x['embedding'] for x in items]
        similarities = [(cosine_sim(current_embedding, embedding), idx) for idx, embedding in enumerate(embedding_candidates)]
        similarities = sorted(similarities, key=lambda x: x[0], reverse=True)
        move_id = item['move_id']
        move_index = item['index']
        moves_data[move_id][move_index]['connections'] =[]
        for (similarity, candidate_index) in similarities:
            moves_data[move_id][move_index]['connections'].append({
                'move_id': items[candidate_index]['move_id'],
                'index': items[candidate_index]['index'],
                'similarity': float(similarity)
            })

# Override the data with moves candidates per step
with open(moves_filepath, 'w') as fp:
    json.dump(moves_data, fp)

In [1]:
import json
import os

moves_filepath = os.path.join('outputs', 'moves.json')
# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    moves_data = json.load(file)

for move_id, data in moves_data.items():
    print(f'{move_id}: {len(data)}')
#moves_data['1841f9d'][8]
#moves_data['5db13cd'][3]

9da1eb8: 19
5d80c9a: 21
8440066: 16
bfb0179: 18
ea90f11: 17
5b961d4: 9
48230c1: 13
29aec7e: 17
fc95277: 15
1841f9d: 16
7254257: 17
acdbcc0: 24
4070a1d: 9
0cced52: 16
3e7415f: 20
a0cfff7: 16
022dccf: 18
5365862: 28
e3cbffd: 17
5508f30: 11
4b8691c: 17
89164f7: 9
fa79fa1: 12
cc02778: 28
09e4474: 11
4abbfb9: 18
824ab00: 15
e7641d8: 9
a4a2022: 9
7fd3943: 15
58adebb: 9
88ae640: 9


In [19]:
#del moves_data['5db13cd']
#moves_data['1841f9d'].pop()
with open(moves_filepath, 'w') as fp:
    json.dump(moves_data, fp)

In [39]:
#moves_data['5db13cd'][17]
#moves_data['1841f9d'][15]['connections'] = moves_data['5d80c9a'][0]['connections']
#moves_data['1841f9d'][15]['connections'][-2]['similarity'] = 1.0
moves_data['7254257'][9]

{'section': 3,
 'count': 1,
 'time': 4.04,
 'leader_keypoints': [[883.8905029296875, 178.8122100830078],
  [885.2084350585938, 172.81875610351562],
  [878.754150390625, 172.350341796875],
  [876.310546875, 176.93077087402344],
  [859.0288696289062, 174.63116455078125],
  [886.4876098632812, 207.85379028320312],
  [816.2603149414062, 212.77598571777344],
  [925.5529174804688, 199.97454833984375],
  [793.5625610351562, 271.8045349121094],
  [934.875732421875, 175.16981506347656],
  [838.3644409179688, 289.9825439453125],
  [868.2261352539062, 317.7515563964844],
  [824.46240234375, 319.8791809082031],
  [917.6929931640625, 387.6554260253906],
  [810.1624755859375, 405.7152404785156],
  [941.8717651367188, 459.5069274902344],
  [774.56884765625, 471.9834899902344]],
 'follower_keypoints': [[883.8905029296875, 178.8122100830078],
  [885.2084350585938, 172.81875610351562],
  [878.754150390625, 172.350341796875],
  [876.310546875, 176.93077087402344],
  [859.0288696289062, 174.63116455078125